# 洪水易感性分析

本notebook用于分析研究区域的洪水易感性，使用机器学习方法基于地形因子和历史洪水数据进行评估。

## 分析流程
1. 数据准备和预处理
2. 特征工程
3. 模型训练和评估
4. 生成易感性地图
5. 结果可视化和分析

In [ ]:
import torch

# 检查 PyTorch 是否安装成功
print("PyTorch 版本:", torch.__version__)

# 检查 GPU 是否可用
print("GPU 可用:", torch.cuda.is_available())

# 检查 CUDA 版本
print("CUDA 版本:", torch.version.cuda)

## 1. 导入必要的库

In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 基础数据处理库
import pandas as pd
import numpy as np
import geopandas as gpd

from sklearn.preprocessing import OneHotEncoder, StandardScaler

# 空间数据处理库
import rasterio
from rasterio.mask import mask

# 可视化库
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 检查GPU是否可用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

## 2. 数据准备

In [22]:
# 定义数据路径
flood_points_file = "Sample/flood_points_utm49n.shp"
non_flood_points_file = "Sample/non_flood_points_utm49n.shp"

# 定义栅格数据路径
raster_files = {
    'dem': 'DEM/DEM_fill_UTM49N.tif',
    'slope': '1RichDem_code/slope_degrees.tif',
    'aspect': '1RichDem_code/aspect.tif',
    'curvature': '1RichDem_code/curvature.tif',
    'profile_curvature': '1RichDem_code/profile_curvature.tif',
    'plan_curvature': '1RichDem_code/planform_curvature.tif',
    'twi': '1RichDem_code/twi.tif',
    'spi': '1RichDem_code/spi.tif',
    'tpi': 'TPI/TopographicPositionIndex.tif',
    'tri': 'TRI/Terrain_Ruggedness_Index.tif',
    'ap': 'AP/AnnualPrecipitation.tif',
    'amdp': 'AMDP/gz_30m_amp(mean_2013-2022).tif',
    'ahrf': 'AHRF/2013-2022_heavy_rain_day(mean)_utm_gz.tif',
    'clcd': 'CLCD/CLCD_2.10.tif',
    'ndvi': 'NDVI/2022_Guangzhou_NDVI_UTM49N.tif',
    'd2r': 'D2R/Dist2River_gz.tif',
    'dd': 'DD/DrainageDensity.tif'
}

In [ ]:
def read_raster_data(raster_path):
    """读取栅格数据并处理NODATA值"""
    with rasterio.open(raster_path) as src:
        array = src.read(1, masked=False)
        nodata = src.nodata
        transform = src.transform
        
        # 处理NODATA值
        if nodata is not None:
            array = np.where(array == nodata, np.nan, array)
            
        return array, transform

def extract_values_at_points_gpu(array, transform, points_gdf, device='cuda'):
    """使用GPU加速栅格值提取"""
    try:
        # 转换为float32以减少内存使用
        array = array.astype(np.float32)
        raster_data = torch.from_numpy(array).to(device)
        
        coords = [(point.x, point.y) for point in points_gdf.geometry]
        rows, cols = zip(*[rasterio.transform.rowcol(transform, x, y) for x, y in coords])
        
        rows = torch.LongTensor(rows).to(device)
        cols = torch.LongTensor(cols).to(device)
        values = raster_data[rows, cols].cpu().numpy()
        
        # 清理GPU内存
        del raster_data
        torch.cuda.empty_cache()
        
        return values
    except Exception as e:
        print(f"Error in extract_values_at_points_gpu: {e}")
        return None

# 读取点数据
try:
    flood_points = gpd.read_file(flood_points_file)
    non_flood_points = gpd.read_file(non_flood_points_file)

    # 添加标签
    flood_points['label'] = 1
    non_flood_points['label'] = 0

    # 合并数据集
    all_points = pd.concat([flood_points, non_flood_points])
    print("点数据读取成功")
except Exception as e:
    print(f"Error reading point data: {e}")

# 提取栅格值
try:
    for factor, path in raster_files.items():
        print(f"处理 {factor}...")
        array, transform = read_raster_data(path)
        values = extract_values_at_points_gpu(array, transform, all_points, device)
        if values is not None:
            all_points[factor] = values
        # 清理内存
        del array
        torch.cuda.empty_cache()
    print("栅格值提取完成")
except Exception as e:
    print(f"Error extracting raster values: {e}")

## 3. 数据探索性分析

In [ ]:
# 定义特征列表
features = ['dem', 'slope', 'aspect', 'curvature', 'profile_curvature', 
           'plan_curvature', 'twi', 'spi', 'tpi', 'tri', 'ap', 'amdp', 'ahrf', 'clcd', 'ndvi', 'd2r', 'dd']

# 准备数据
X = all_points[features].values.astype(np.float32)

# 处理无效值
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
X = torch.FloatTensor(X)
y = torch.LongTensor(all_points['label'].values)

# 数据标准化
eps = 1e-8  # 添加小的常数避免除零
mean = X.mean(dim=0)
std = X.std(dim=0) + eps
X = (X - mean) / std

# 显示数据集基本信息
print("数据集形状:", all_points.shape)
print("\n特征列表:")
print(all_points.columns.tolist())
print("\n特征统计描述:")
print(all_points[features].describe())

# 假设 all_points 是你的数据集
missing_values = all_points[features].isna().sum()
print(missing_values)

In [ ]:
def handle_missing_data(all_points, features):
    """
    处理缺失数据（修复数据类型问题）
    """
    print("\n开始处理缺失数据...")
    data = all_points.copy()
    
    # 确保数值型列的类型为float32
    numerical_features = [f for f in features if f != 'clcd']
    for feature in numerical_features:
        data[feature] = data[feature].astype('float32')
    
    # 处理缺失值
    for feature in features:
        missing_count = data[feature].isnull().sum()
        if missing_count > 0:
            print(f"\n处理 {feature} 的缺失值 ({missing_count} 个)...")
            
            if feature == 'clcd':
                # 使用loc替代fillna的inplace操作
                mode_value = data[feature].mode()[0]
                data.loc[data[feature].isnull(), feature] = mode_value
                
            else:
                missing_ratio = missing_count / len(data)
                
                if missing_ratio < 0.05:
                    # 使用相邻点的平均值填充
                    mask = data[feature].isnull()
                    coords = np.array([(p.x, p.y) for p in data.geometry])
                    
                    for idx in data[mask].index:
                        current_point = coords[idx]
                        distances = np.sqrt(np.sum((coords - current_point)**2, axis=1))
                        
                        valid_mask = ~data[feature].isnull()
                        valid_distances = distances[valid_mask]
                        valid_values = data[feature][valid_mask].values
                        
                        k = min(5, len(valid_values))
                        nearest_indices = np.argpartition(valid_distances, k)[:k]
                        
                        weights = 1 / (valid_distances[nearest_indices] + 1e-6)
                        weighted_mean = np.average(
                            valid_values[nearest_indices], 
                            weights=weights
                        ).astype('float32')
                        
                        data.loc[idx, feature] = weighted_mean
                        
                elif missing_ratio < 0.2:
                    # 使用特征相关性填充
                    corr = data[features].corr()[feature].abs()
                    best_feature = corr.sort_values(ascending=False).index[1]
                    
                    train_mask = ~data[feature].isnull()
                    X_train = data[best_feature][train_mask].values.reshape(-1, 1)
                    y_train = data[feature][train_mask].values
                    
                    lr = LinearRegression()
                    lr.fit(X_train, y_train)
                    
                    X_pred = data[best_feature][~train_mask].values.reshape(-1, 1)
                    predicted_values = lr.predict(X_pred).astype('float32')
                    data.loc[~train_mask, feature] = predicted_values
                    
                else:
                    # 使用中位数填充
                    median_value = data[feature].median()
                    data.loc[data[feature].isnull(), feature] = median_value
    
    # 验证是否还有缺失值
    remaining_missing = data[features].isnull().sum()
    if remaining_missing.any():
        print("\n警告：仍存在缺失值:")
        print(remaining_missing[remaining_missing > 0])
    else:
        print("\n所有缺失值已处理完成")
    
    return data
try:
    all_points = handle_missing_data(all_points, features)
except Exception as e:
    print(f"处理过程出错: {e}")

In [ ]:
def validate_processed_data(data, features, original_data=None):
    """
    验证处理后的数据质量
    
    参数:
        data: 处理后的DataFrame
        features: 特征列表
        original_data: 原始DataFrame（可选）
    """
    print("\n=== 数据处理后验证 ===")
    
    # 1. 基本检查
    print("\n1. 基本数据检查:")
    print(f"样本数量: {len(data)}")
    print(f"特征数量: {len(features)}")
    
    # 2. 检查缺失值
    missing = data[features].isnull().sum()
    if missing.any():
        print("\n2. 发现缺失值:")
        print(missing[missing > 0])
    else:
        print("\n2. 无缺失值 ✓")
    
    # 3. 检查数值范围
    print("\n3. 数值范围检查:")
    for feature in features:
        if feature != 'clcd':  # 排除类别型数据
            values = data[feature]
            print(f"\n{feature}:")
            print(f"  范围: [{values.min():.2f}, {values.max():.2f}]")
            print(f"  均值: {values.mean():.2f}")
            print(f"  标准差: {values.std():.2f}")
            
            # 检查异常值
            q1 = values.quantile(0.25)
            q3 = values.quantile(0.75)
            iqr = q3 - q1
            outliers = values[(values < (q1 - 1.5 * iqr)) | (values > (q3 + 1.5 * iqr))]
            if len(outliers) > 0:
                print(f"  异常值数量: {len(outliers)} ({(len(outliers)/len(values)*100):.2f}%)")
    
    # 4. 检查类别分布
    if 'clcd' in features:
        print("\n4. CLCD类别分布:")
        print(data['clcd'].value_counts())
    
    # 5. 如果有原始数据，比较处理前后的变化
    if original_data is not None:
        print("\n5. 处理前后对比:")
        for feature in features:
            if feature != 'clcd':
                orig_mean = original_data[feature].mean()
                proc_mean = data[feature].mean()
                change_pct = ((proc_mean - orig_mean) / orig_mean * 100) if orig_mean != 0 else 0
                print(f"\n{feature}:")
                print(f"  原始均值: {orig_mean:.2f}")
                print(f"  处理后均值: {proc_mean:.2f}")
                print(f"  变化百分比: {change_pct:.2f}%")
    
    # 6. 特征相关性检查
    print("\n6. 特征相关性检查:")
    corr_matrix = data[features].corr()
    high_corr = np.where(np.abs(corr_matrix) > 0.8)
    high_corr = [(features[i], features[j], corr_matrix.iloc[i, j]) 
                 for i, j in zip(*high_corr) if i < j]
    if high_corr:
        print("\n发现高相关特征:")
        for f1, f2, corr in high_corr:
            print(f"{f1} - {f2}: {corr:.2f}")
    
    # # 7. 可视化处理后的分布
    #  plt.figure(figsize=(15, 5))
    # for i, feature in enumerate(features[:3]):  # 展示前三个特征作为示例
    #     if feature != 'clcd':
    #         plt.subplot(1, 3, i+1)
    #         sns.histplot(data=data, x=feature, hue='label', bins=30)
    #         plt.title(f'{feature} 分布')
    # plt.tight_layout()
    # plt.show()

# 在主处理流程中使用
try:
    # 保存原始数据的副本
    original_data = all_points.copy()
    
    # 处理缺失数据
    all_points = handle_missing_data(all_points, features)
    
    # 验证处理后的数据
    validate_processed_data(all_points, features, original_data)
    
    # ... 继续后续处理 ...
    
except Exception as e:
    print(f"处理过程出错: {e}")

## 5. 特征重要性分析

In [ ]:
def preprocess_data(all_points, features):
    """
    数据预处理：包括分类变量编码、数值标准化和空间特征提取
    """
    print("\n开始数据预处理...")
    
    # 1. 分离类别型和数值型特征
    categorical_features = ['clcd']
    numerical_features = [f for f in features if f not in categorical_features]
    
    # 2. 处理类别型特征
    print("\n处理类别型特征...")
    enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    categorical_data = all_points[categorical_features].values
    categorical_encoded = enc.fit_transform(categorical_data)
    
    # 获取编码后的特征名称
    categorical_names = [f'clcd_{i+1}' for i in range(categorical_encoded.shape[1])]
    
    # 3. 处理数值型特征
    print("\n处理数值型特征...")
    numerical_data = all_points[numerical_features].values.astype(np.float32)
    
    # 4. 提取空间特征
    print("\n提取空间特征...")
    coords = np.column_stack([
        all_points.geometry.x,
        all_points.geometry.y
    ])
    
    # 5. 标准化数值特征和空间特征
    print("\n特征标准化...")
    scaler = StandardScaler()
    numerical_scaled = scaler.fit_transform(numerical_data)
    coords_scaled = scaler.fit_transform(coords)
    
    # 6. 合并所有特征
    X = np.column_stack([
        numerical_scaled,    # 标准化的数值特征
        categorical_encoded, # 独热编码的类别特征
        coords_scaled       # 标准化的空间特征
    ])
    
    # 7. 准备特征名称列表
    feature_names = (
        numerical_features +    # 数值特征名称
        categorical_names +     # 类别特征编码后的名称
        ['x_coord', 'y_coord'] # 空间特征名称
    )
    
    # 8. 准备标签
    y = all_points['label'].values
    
    # 9. 创建处理后的DataFrame
    processed_df = pd.DataFrame(X, columns=feature_names)
    processed_df['label'] = y
    
    print("\n数据预处理完成!")
    print(f"处理后特征数量: {len(feature_names)}")
    print(f"样本数量: {len(processed_df)}")
    
    return processed_df, X, y, scaler, enc
def validate_preprocessing(processed_df, original_df, feature_names):
    """
    验证预处理结果
    """
    print("\n=== 预处理验证 ===")
    
    # 1. 检查缺失值
    missing = processed_df.isnull().sum()
    if missing.any():
        print("\n警告: 存在缺失值:")
        print(missing[missing > 0])
    else:
        print("\n✓ 无缺失值")
    
    # 2. 检查数值范围
    print("\n数值特征统计:")
    print(processed_df.describe())
    
    # 3. 检查类别编码
    if 'clcd' in original_df.columns:
        print("\n类别编码验证:")
        print("原始CLCD类别:", original_df['clcd'].unique())
        clcd_cols = [col for col in processed_df.columns if col.startswith('clcd_')]
        print("编码后的CLCD列:", clcd_cols)
    
    # 4. 检查空间特征
    print("\n空间特征范围:")
    print(processed_df[['x_coord', 'y_coord']].describe())
# 使用示例
try:
    # 假设 all_points 是你的数据集
    # 定义特征列表
    features = ['dem', 'slope', 'aspect', 'curvature', 'profile_curvature', 
               'plan_curvature', 'twi', 'spi', 'tpi', 'tri', 'ap', 'amdp', 
               'ahrf', 'clcd', 'ndvi', 'd2r', 'dd']
    
    # 数据预处理
    processed_df, X, y, scaler, encoder = preprocess_data(all_points, features)
    
    # 验证预处理结果
    validate_preprocessing(processed_df, all_points, features)
    
    print("\n预处理完成，可以进行GeoXGBoost模型训练")
    
except Exception as e:
    print(f"预处理过程出错: {e}")

## 6. 生成易感性地图